#Loading Trained Data
This data was trained by Encore Alert team in the business classification trainner on http://encorealert.com:8088

## File Load

In [2]:
import pandas as pd
import glob

trained_files = glob.glob("data/trained/twitter-actor.csv*")
trained_dataframes = []
for file in trained_files:
    df = pd.read_csv(file, header=None, names=['native_id', 'class'])
    trained_dataframes.append(df)
    print('# Parsed', file, 'with', len(df), 'lines.')
trained_data = pd.concat(trained_dataframes)

# Parsed data/trained/twitter-actor.csv with 135 lines.
# Parsed data/trained/twitter-actor.csv.20150606 with 2 lines.
# Parsed data/trained/twitter-actor.csv.20150607 with 309 lines.
# Parsed data/trained/twitter-actor.csv.20150608 with 59 lines.
# Parsed data/trained/twitter-actor.csv.20150609 with 3 lines.
# Parsed data/trained/twitter-actor.csv.20150610 with 1039 lines.
# Parsed data/trained/twitter-actor.csv.20150612 with 468 lines.


In [3]:
trained_data.head()

,native_id,class
0,427078784064294912,personal
1,427115022842867713,business
2,427244265346195456,business
3,427334246957473792,personal
4,427353217144336384,personal


In [6]:
trained_data.groupby('class').count()

,native_id
class,
bot,42
business,874
personal,1037
spam,62


In [19]:
### Filter only the classes that we have minimal amount of data
# In this example, there is not enough bots, so ignoring them from now

trained_data = trained_data[trained_data['class'].isin(['business', 'personal'])]
trained_data = trained_data.drop_duplicates('native_id')
trained_data['native_id'] = trained_data['native_id'].astype('str')
trained_data.groupby('class').count()

,native_id
class,
business,603
personal,751


## Using Twitter to retrieve metadata
The scraped data consist mainly of handles and categories, so we need to retrieve extra actor information for trainning

In [20]:
from twitter import *

token = '22911906-GR7LBJ2oil3cc27aUIAln4zur4F7CdKAKyEi6NDzi'
token_key = 'FZbyPm1i3BMfiXKlKPuzBdRlvbenW09n8LX5OvgM85g'
con_secret = 'cyZ6NLdySvTkhKGUGmXMKw'
con_secret_key = '5UgOJOanohNPMVkfLY85CjzdMcNAAVBlRCyGYys'

t = Twitter(auth=OAuth(token, token_key, con_secret, con_secret_key))

In [46]:
import numpy as np
import time
import sys

def lookup_twitter_statuses(data):
    # Get trained_data buckets too lookup
    indices = np.arange(len(data))
    max_mod = int((len(data)/100)+1)
    tweets = []
    for x in range(0, max_mod):
        native_ids = data[(indices % max_mod) == x]['native_id']
        tweets += t.statuses.lookup(_id=','.join(native_ids), _timeout=3)
        print('Downloaded tweet info for', len(tweets))
    return tweets

In [42]:
import ast
import re
from html import escape

def persist_complete_data(file_name, data):
    
    s = 0
    with open(file_name, 'w') as csv_file:
        tweets_writer = csv.writer(csv_file)
        tweets_writer.writerow([
            'actor_id',
            'actor_screen_name',
            'actor_name',
            'actor_verified',
            'actor_friends_count',
            'actor_followers_count',
            'actor_listed_count',
            'actor_statuses_count',
            'actor_favorites_count',
            'actor_summary',
            'actor_created_at',
            'actor_location',

            'tweet_id',
            'tweet_created_at',
            'tweet_generator',
            'tweet_body',
            'tweet_verb',

            'tweet_urls_count',
            'tweet_mentions_count',
            'tweet_hashtags_count',
            'tweet_trends_count',
            'tweet_symbols_count'])
        for tweet in data:
            tweets_writer.writerow([
                tweet['user']['id'],
                tweet['user']['screen_name'],
                tweet['user']['name'],
                tweet['user']['verified'],
                tweet['user']['friends_count'],
                tweet['user']['followers_count'],
                tweet['user']['listed_count'],
                tweet['user']['statuses_count'],
                tweet['user']['favourites_count'],
                tweet['user']['description'],
                tweet['user']['created_at'],
                tweet['user']['location'] if tweet['user'].get('location') else 'null',

                tweet['id'],
                tweet['created_at'],
                re.findall('>(.*)<', tweet['source'])[0],
                tweet['text'],
                not tweet['retweeted'],
                len(tweet['entities']['urls']),
                len(tweet['entities']['user_mentions']),
                len(tweet['entities']['hashtags']),
                "",
                len(tweet['entities']['symbols'])
            ])
            s += 1
            if (s % 100 == 0): print('Already writed', s, 'rows to', file_name)

In [47]:
business_data = lookup_twitter_statuses(trained_data[trained_data['class'] == 'business'])
persist_complete_data('data/csv/brand_manually_trained.csv', business_data)

Downloaded tweet info for 87
Downloaded tweet info for 173
Downloaded tweet info for 258
Downloaded tweet info for 341
Downloaded tweet info for 426
Downloaded tweet info for 512
Downloaded tweet info for 597
Already writed 100 rows to data/csv/manually_trained_business.csv
Already writed 200 rows to data/csv/manually_trained_business.csv
Already writed 300 rows to data/csv/manually_trained_business.csv
Already writed 400 rows to data/csv/manually_trained_business.csv
Already writed 500 rows to data/csv/manually_trained_business.csv


In [50]:
personal_data = lookup_twitter_statuses(trained_data[trained_data['class'] == 'personal'])
persist_complete_data('data/csv/person_manually_trained.csv', personal_data)

Already writed 100 rows to data/csv/manually_trained_personal.csv
Already writed 200 rows to data/csv/manually_trained_personal.csv
Already writed 300 rows to data/csv/manually_trained_personal.csv
Already writed 400 rows to data/csv/manually_trained_personal.csv
Already writed 500 rows to data/csv/manually_trained_personal.csv
Already writed 600 rows to data/csv/manually_trained_personal.csv
Already writed 700 rows to data/csv/manually_trained_personal.csv
